In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [3]:
# missing date 추가
folder = 'data'
#file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx']]
file_name0 = '가평_2019.xlsx'
#file_name1 = '해평_2016.xlsx'
file_name1 = '도개_2019.xlsx'
#file_name1 = '의암호_2017.xlsx'

path = os.path.join(folder, file_name0)
df0 = pd.read_excel(path)
path = os.path.join(folder, file_name1)
df1 = pd.read_excel(path)
print(df0.shape)
print(df1.shape)

date_full = df0.iloc[:, 0]
fill = { col: np.nan for col in df1.columns}

j = 0
for i in range(len(date_full)):
    #print(i, date_full[i])
    #print(df1.iloc[j])
    #print(df1.iloc[j,0])
    while df1.iloc[j, 0][5:10] == '02.29':
        j += 1
    if df1.iloc[j, 0][5:] != date_full[i][5:]:
        print(i, date_full[i], j, df1.iloc[j,0])
        fill_date = df1.iloc[j,0][:5] + date_full[i][5:]
        #print(fill_date)
        fill[df1.columns[0]] = fill_date
        #print(fill)
        line = pd.DataFrame(fill, index=[j-0.5])
        df1 = df1.append(line, ignore_index=False)
        df1 = df1.sort_index().reset_index(drop=True)
    j += 1
print(i+1,j)
df1.to_excel(path, index=False)


(8760, 12)
(8760, 12)
8760 8760


In [ ]:
df1

In [ ]:
df1

In [ ]:
df0

**한강 데이터 로딩**

In [4]:
folder = 'data'
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx']]
#file_names = [['해평_2016.xlsx','해평_2017.xlsx','해평_2018.xlsx', '해평_2019.xlsx'], ['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx']]
#file_names = [['가평_2019.xlsx'], ['의암호_2019.xlsx']]

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx


/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx


**낙동강 해평-도개**

In [4]:
folder = 'data'
#file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx']]
file_names = [['해평_2016.xlsx','해평_2017.xlsx','해평_2018.xlsx', '해평_2019.xlsx'], ['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx']]
#file_names = [['가평_2019.xlsx'], ['의암호_2019.xlsx']]

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    if loc == 0:
        df.append(df_full[loc].iloc[:, 2:9].copy())
    else:
        df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]].copy())
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

해평_2016.xlsx
해평_2017.xlsx
해평_2018.xlsx
해평_2019.xlsx
도개_2016.xlsx
도개_2017.xlsx
도개_2018.xlsx
도개_2019.xlsx


In [5]:
df_full

[                  측정날짜 측정소명   수온  수소이온농도  전기전도도  용존산소   탁도  총유기탄소  클로로필-a  \
 0     2016.01.01 00:00   해평  4.5     7.4  335.0  11.8  3.9    1.8     9.9   
 1     2016.01.01 01:00   해평  4.5     7.4  334.0  11.8  3.8    1.9    10.0   
 2     2016.01.01 02:00   해평  4.5     7.4  334.0  11.7  3.8    1.9     9.9   
 3     2016.01.01 03:00   해평  4.5     7.4  334.0  11.7  3.8    1.8    10.2   
 4     2016.01.01 04:00   해평  4.5     7.4  336.0  11.7  3.8    1.9    10.0   
 ...                ...  ...  ...     ...    ...   ...  ...    ...     ...   
 8755  2019.12.31 19:00   해평  4.2     7.8  260.0  12.8  1.5    2.6    17.6   
 8756  2019.12.31 20:00   해평  4.2     7.8  261.0  12.8  1.5    2.6    17.6   
 8757  2019.12.31 21:00   해평  4.1     7.8  261.0  12.8  1.5    2.6    17.6   
 8758  2019.12.31 22:00   해평  4.1     7.8  261.0  12.8  1.5    2.6    17.6   
 8759  2019.12.31 23:00   해평  4.0     7.8  261.0  12.8  1.5    2.6    17.7   
 
       염화메틸렌  ...  [ECD]트리클로로에틸렌  [ECD]테트라클로로에틸렌   페놀  독성지수(좌)

In [ ]:
df[1]


**Normalize**

In [6]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [7]:
print(df_all.shape)

(70128, 11)


In [8]:
train_mean, train_std

(수온          1.620714e+01
 수소이온농도      7.906945e+00
 전기전도도       2.538927e+02
 용존산소        1.024297e+01
 탁도          6.886673e+00
 총유기탄소       2.788176e+00
 클로로필-a      1.683803e+01
 Day sin     6.037104e-15
 Day cos     2.747637e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           9.030900
 수소이온농도       0.520447
 전기전도도       40.167703
 용존산소         2.562898
 탁도          12.328697
 총유기탄소        0.860476
 클로로필-a      13.111239
 Day sin      0.707112
 Day cos      0.707112
 Year sin     0.707105
 Year cos     0.707119
 dtype: float64)

In [9]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.296343,-0.974057,2.019218,0.607528,-0.242254,-1.148407,-0.529166,-1.999581e-12,1.414203,-0.003771,1.414155
1,-1.296343,-0.974057,1.994322,0.607528,-0.250365,-1.032192,-0.521539,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.296343,-0.974057,1.994322,0.568510,-0.250365,-1.032192,-0.529166,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.296343,-0.974057,1.994322,0.568510,-0.250365,-1.148407,-0.506285,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.296343,-0.974057,2.044113,0.568510,-0.250365,-1.032192,-0.521539,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.329562,-0.205486,0.152046,0.997712,-0.436921,-0.218688,0.058116,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.329562,-0.205486,0.176942,0.997712,-0.436921,-0.218688,0.058116,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.340635,-0.205486,0.176942,0.997712,-0.436921,-0.218688,0.058116,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.340635,-0.205486,0.176942,0.997712,-0.436921,-0.218688,0.058116,-7.071017e-01,1.224736,-0.005068,1.414151


In [10]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [11]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [12]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [13]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [14]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [15]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

**WindowGenerator 테스트**

In [14]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=None)
w2

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): None

In [15]:
w1 = WindowGenerator(input_width=24, label_width=1, shift=1,
                     label_columns='수온')
w1

Total window size: 25
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24]
Label column name(s): 수온

In [16]:
# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])


example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 7, 11)
Inputs shape: (3, 6, 11)
labels shape: (3, 1, 11)


In [17]:
w2.example = example_inputs, example_labels

In [18]:
w2.plot(plot_col='수온')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Window Generator trainset 정의**

In [16]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [17]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [18]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [19]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [20]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [21]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

**miss_data 테스트**

In [22]:
norm_df = pd.concat(df,axis=0)
n_data = norm_df.to_numpy()
MissData.save(n_data, max_tseq=12)
n_data
#n_data = n_data[0:100]
isnan = np.isnan(n_data).astype(int)
isnan[50:100]
miss = MissData(load_dir='save')
tt = miss.make_missdata(n_data)
tt = np.isnan(tt).astype(int)
tt[3000:3050]

miss_data file saved


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

**miss data 준비**

In [23]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp + 0.5*(1-H_temp)
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [364]:
from tensorflow.keras.layers import Conv1D

class GAIN_cnn(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN_cnn, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        shape = self.shape
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')

        a = Concatenate()([x, m])

        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        #a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        #a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        #G_prob = Conv1D(filters=shape[1], kernel_size=(7,), padding='same', activation=last_activation, kernel_initializer=xavier_initializer)(a)
        a = Dense(shape[-1]*2, activation='relu', kernel_initializer=xavier_initializer)(a)
        G_prob = Dense(shape[-1], activation=last_activation, kernel_initializer=xavier_initializer)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')

        a = Concatenate()([x, h])
        
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)
        a = Conv1D(filters=shape[1]*2, kernel_size=(7,), padding='same', activation='relu')(a)

        #a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #D_prob = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        #D_prob = Conv1D(filters=shape[1], kernel_size=(7,), padding='same', activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        a = Dense(shape[-1]*2, activation='relu', kernel_initializer=xavier_initializer)(a)
        D_prob = Dense(shape[-1], activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        #dims = np.prod(shape[1:])
        #input_width = shape[1]
        # print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_pred)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

**GAIN 모델 test**

In [25]:
gain = GAIN(shape=(2,7))
gain.compile(loss=GAIN.RMSE_loss)
#gain_cnn = GAIN_cnn(shape=(2,7))
#gain_cnn.compile(loss=GAIN.RMSE_loss)

In [ ]:
x = np.random.random((1,2,7))
x[0][0][2] = np.nan
m = np.random.random((1,2,7))
#y = gain_cnn.generator([x, m])
y = gain.generator.predict([x, m])
print(y.shape)
y = gain.discriminator.predict([x,m])
y.shape
print(x.shape)
y = gain.predict(x)

In [ ]:
gain.fit(x,y)
#gain_cnn.fit(x,y)

## spam data gain 학습 테스트

In [ ]:
df_spam = pd.read_csv('data/spam.csv')
dg_spam = GainDataGenerator([df_spam], batch_size=128, input_width=1, label_width=1)
it = iter(dg_spam)
x,y = next(it)
print(dg_spam.shape)
x.shape, y.shape

In [ ]:
model = GAIN(shape=dg_spam.shape[1:])
model.compile(loss=GAIN.RMSE_loss)

In [ ]:
#model = GAIN_cnn(shape=dg_spam.shape[1:])
#model.compile(loss=GAIN.RMSE_loss)

In [ ]:
model.fit(dg_spam, batch_size=128, epochs=30)
#model.fit(x, y, batch_size=128)
#model.fit(dg_spam, batch_size=4601, epochs=1)

In [ ]:
x = dg_spam.data.copy()
y = dg_spam.data.copy()
m = dg_spam.data_m
x[m == 0] = np.nan
y[m == 1] = np.nan
x = x.reshape(x.shape[0], 1, x.shape[1])
y = y.reshape(y.shape[0], 1, y.shape[1])
x.shape


#model.fit(x,y)

In [ ]:
#model.load()

**spam data rmse 측정**

In [ ]:
print(y.shape)
ret = model.evaluate(x, y)
print(ret)

In [ ]:
x_input = x[0:4601]
y_true = y[0:4601]
y_pred = model.predict(x_input)
#print(x_input)
#print(y_true)
#print(y_pred)
isnan = np.isnan(y_true)
diff = y_pred - y_true
diff[isnan] = 0.
#print(diff)
m = isnan.astype(int)
n = np.sum(1-m)
rmse = np.sqrt(np.sum(diff**2)/float(n))
print('rmse =', rmse)

In [ ]:
model.summary()

**spam data dataset으로 학습하기**

In [ ]:
ds = tf.data.Dataset.from_generator(
  lambda: dg_spam,
  output_types=(tf.float32, tf.float32),
  output_shapes=(
    dg_spam.shape,
    dg_spam.shape
    #[batch_size, train_generator.dim],
    #[batch_size, train_generator.dim],
  )
).repeat(-1).prefetch(10)

In [ ]:
it = iter(ds)
x,y = next(it)
x.shape, y.shape

In [ ]:
history = model.fit(ds, steps_per_epoch=10, epochs=1000)

**학습성능 측정(rsme)**

In [ ]:
model.evaluate(ds, steps=50)

**학습 그래프**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
#ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

# 수질 GAIN 데이터

**데이터 준비**

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [28]:
train_df = df_all
val_df = df_all
test_df = df_all

In [29]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

_ = wide_window.train

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

**학습용 데이터 plotting**

In [30]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,-0.468079,-1.999581e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,-0.496459,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,-0.496459,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,-0.496459,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,-0.496459,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,1.140110,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,1.064430,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,1.007671,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,1.026590,-7.071017e-01,1.224736,-0.005068,1.414151


In [267]:
df[1]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.274197,-0.205486,1.969427,0.334400,1.266422,-1.613266,-0.773232,-1.999581e-12,1.414203,-0.003771,1.414155
1,-1.274197,-0.205486,1.969427,0.334400,1.266422,-1.729481,-0.765605,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.274197,-0.205486,1.969427,0.295382,1.274533,-1.729481,-0.750351,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.274197,-0.205486,1.969427,0.295382,1.266422,-1.729481,-0.750351,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.274197,-0.205486,1.969427,0.295382,1.266422,-1.613266,-0.742724,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.373855,0.370942,0.326315,-0.055783,-0.307143,-0.683548,3.147069,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.373855,0.370942,0.276524,-0.055783,-0.307143,-0.567333,3.200458,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.373855,0.370942,0.226733,-0.094802,-0.315254,-0.567333,3.093680,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.396001,0.370942,0.251629,-0.094802,-0.315254,-0.683548,3.116561,-7.071017e-01,1.224736,-0.005068,1.414151


In [30]:
wide_window.plot(plot_col='클로로필-a')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  17210


In [31]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:7], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습

In [33]:
val_performance = {}
performance = {}

In [34]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [32]:
#gain = GAIN_cnn(shape=wide_window.dg.shape[1:], gen_sigmoid=False, alpha=200.)
#gain.compile(loss=GAIN.RMSE_loss)

In [35]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 불러오기(사전 학습데이터)**

In [34]:
#model.fit를 사용하지 않을 때에는 학습 데이터 로딩
#gain.load(save_dir='save')
gain.load(save_dir='save_nagdong')

model weights loaded


**모델 학습**

In [36]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)


#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                    patience=2,
#                                                    mode='min')
#gain.compile()


Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 120.3892 - disc_loss: 0.7253 - rmse: 1.0379 - val_loss: 0.8922
Epoch 2/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 94.8459 - disc_loss: 0.5360 - rmse: 0.9145 - val_loss: 0.8126
Epoch 3/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 79.3213 - disc_loss: 0.4716 - rmse: 0.8607 - val_loss: 0.7366
Epoch 4/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 70.8652 - disc_loss: 0.4208 - rmse: 0.8173 - val_loss: 0.7564
Epoch 5/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 61.2067 - disc_loss: 0.3847 - rmse: 0.7735 - val_loss: 0.8055
Epoch 6/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 52.8912 - disc_loss: 0.3901 - rmse: 0.7107 - val_loss: 0.6505
Epoch 7/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 45.2564 - disc_loss: 0.3949 - rmse: 0.6709 - val_loss: 0.6426
Epoch 8/2000
1/1 [====

1/1 [==============================] - 0s 130ms/step - gen_loss: 4.4567 - disc_loss: 0.2326 - rmse: 0.3059 - val_loss: 0.2565
Epoch 118/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 3.2654 - disc_loss: 0.2322 - rmse: 0.2492 - val_loss: 0.2679
Epoch 119/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 3.7176 - disc_loss: 0.2363 - rmse: 0.2701 - val_loss: 0.2844
Epoch 120/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 3.2102 - disc_loss: 0.2304 - rmse: 0.2266 - val_loss: 0.2704
Epoch 121/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 3.4112 - disc_loss: 0.2260 - rmse: 0.2322 - val_loss: 0.2511
Epoch 122/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 4.8817 - disc_loss: 0.2359 - rmse: 0.2769 - val_loss: 0.3089
Epoch 123/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 3.3121 - disc_loss: 0.2327 - rmse: 0.2546 - val_loss: 0.3094
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 117ms/step - gen_loss: 3.2635 - disc_loss: 0.1990 - rmse: 0.2573 - val_loss: 0.2720
Epoch 234/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 2.3613 - disc_loss: 0.1956 - rmse: 0.2123 - val_loss: 0.2331
Epoch 235/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 3.3870 - disc_loss: 0.1908 - rmse: 0.2665 - val_loss: 0.2712
Epoch 236/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 2.7827 - disc_loss: 0.1998 - rmse: 0.2195 - val_loss: 0.3074
Epoch 237/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.6290 - disc_loss: 0.1898 - rmse: 0.2303 - val_loss: 0.2600
Epoch 238/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 3.0669 - disc_loss: 0.1958 - rmse: 0.3054 - val_loss: 0.2516
Epoch 239/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 2.8048 - disc_loss: 0.1942 - rmse: 0.2336 - val_loss: 0.2546
Epoch 240/2000
1/1 [========

1/1 [==============================] - 0s 152ms/step - gen_loss: 2.4436 - disc_loss: 0.1782 - rmse: 0.3644 - val_loss: 0.2478
Epoch 350/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 2.6825 - disc_loss: 0.1728 - rmse: 0.3397 - val_loss: 0.2637
Epoch 351/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.8337 - disc_loss: 0.1708 - rmse: 0.4664 - val_loss: 0.2877
Epoch 352/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 2.4772 - disc_loss: 0.1705 - rmse: 0.2089 - val_loss: 0.2549
Epoch 353/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.5164 - disc_loss: 0.1742 - rmse: 0.2048 - val_loss: 0.2442
Epoch 354/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 2.5478 - disc_loss: 0.1737 - rmse: 0.2086 - val_loss: 0.2586
Epoch 355/2000
1/1 [==============================] - 0s 103ms/step - gen_loss: 2.5768 - disc_loss: 0.1773 - rmse: 0.2114 - val_loss: 0.2246
Epoch 356/2000
1/1 [========

1/1 [==============================] - 0s 94ms/step - gen_loss: 2.7252 - disc_loss: 0.1501 - rmse: 0.2295 - val_loss: 0.2429
Epoch 466/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 2.3209 - disc_loss: 0.1504 - rmse: 0.2597 - val_loss: 0.2119
Epoch 467/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 2.3081 - disc_loss: 0.1545 - rmse: 0.2216 - val_loss: 0.2213
Epoch 468/2000
1/1 [==============================] - 0s 94ms/step - gen_loss: 3.0101 - disc_loss: 0.1527 - rmse: 0.2296 - val_loss: 0.2250
Epoch 469/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 2.2160 - disc_loss: 0.1526 - rmse: 0.2474 - val_loss: 0.2465
Epoch 470/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.8931 - disc_loss: 0.1519 - rmse: 0.2174 - val_loss: 0.2898
Epoch 471/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 2.1750 - disc_loss: 0.1432 - rmse: 0.2433 - val_loss: 0.2415
Epoch 472/2000
1/1 [==========

1/1 [==============================] - 0s 101ms/step - gen_loss: 2.9217 - disc_loss: 0.1419 - rmse: 0.2125 - val_loss: 0.2141
Epoch 582/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.0597 - disc_loss: 0.1444 - rmse: 0.1921 - val_loss: 0.2146
Epoch 583/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 2.0961 - disc_loss: 0.1417 - rmse: 0.2182 - val_loss: 0.2290
Epoch 584/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 2.3999 - disc_loss: 0.1437 - rmse: 0.2207 - val_loss: 0.2416
Epoch 585/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.4893 - disc_loss: 0.1410 - rmse: 0.2317 - val_loss: 0.2442
Epoch 586/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.6049 - disc_loss: 0.1385 - rmse: 0.2176 - val_loss: 0.2176
Epoch 587/2000
1/1 [==============================] - 0s 96ms/step - gen_loss: 2.1293 - disc_loss: 0.1335 - rmse: 0.2141 - val_loss: 0.2118
Epoch 588/2000
1/1 [=========

1/1 [==============================] - 0s 158ms/step - gen_loss: 2.2082 - disc_loss: 0.1378 - rmse: 0.2052 - val_loss: 0.2107
Epoch 698/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.2846 - disc_loss: 0.1354 - rmse: 0.2165 - val_loss: 0.2220
Epoch 699/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 2.0532 - disc_loss: 0.1369 - rmse: 0.1999 - val_loss: 0.2308
Epoch 700/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 2.1435 - disc_loss: 0.1329 - rmse: 0.2376 - val_loss: 0.2222
Epoch 701/2000
1/1 [==============================] - 0s 89ms/step - gen_loss: 2.4839 - disc_loss: 0.1376 - rmse: 0.2492 - val_loss: 0.2137
Epoch 702/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 2.0254 - disc_loss: 0.1330 - rmse: 0.2110 - val_loss: 0.2092
Epoch 703/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.4965 - disc_loss: 0.1422 - rmse: 0.2165 - val_loss: 0.1990
Epoch 704/2000
1/1 [=========

1/1 [==============================] - 0s 136ms/step - gen_loss: 1.9504 - disc_loss: 0.1284 - rmse: 0.1996 - val_loss: 0.2184
Epoch 814/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.3213 - disc_loss: 0.1282 - rmse: 0.3889 - val_loss: 0.3824
Epoch 815/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 2.3242 - disc_loss: 0.1255 - rmse: 0.2268 - val_loss: 0.2095
Epoch 816/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 2.3764 - disc_loss: 0.1269 - rmse: 0.3104 - val_loss: 0.4827
Epoch 817/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.1225 - disc_loss: 0.1248 - rmse: 0.2068 - val_loss: 0.2357
Epoch 818/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 2.3088 - disc_loss: 0.1271 - rmse: 0.2600 - val_loss: 0.3861
Epoch 819/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.2822 - disc_loss: 0.1286 - rmse: 0.2008 - val_loss: 0.3568
Epoch 820/2000
1/1 [========

1/1 [==============================] - 0s 136ms/step - gen_loss: 1.8918 - disc_loss: 0.1111 - rmse: 0.1937 - val_loss: 0.2097
Epoch 930/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 1.7878 - disc_loss: 0.1200 - rmse: 0.3678 - val_loss: 0.2084
Epoch 931/2000
1/1 [==============================] - 0s 95ms/step - gen_loss: 2.1126 - disc_loss: 0.1146 - rmse: 0.2366 - val_loss: 0.2148
Epoch 932/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 1.9716 - disc_loss: 0.1146 - rmse: 0.2021 - val_loss: 0.2491
Epoch 933/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 1.7975 - disc_loss: 0.1165 - rmse: 0.2924 - val_loss: 0.2558
Epoch 934/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 1.9772 - disc_loss: 0.1184 - rmse: 0.2071 - val_loss: 0.5063
Epoch 935/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.9910 - disc_loss: 0.1210 - rmse: 0.2038 - val_loss: 0.2332
Epoch 936/2000
1/1 [=========

In [37]:
#gain.save(save_dir='save')
#gain.save(save_dir='save_paldang')
gain.save(save_dir='save_nagdong')

**학습 loss history 출력**

In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [39]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 3s 25ms/step - loss: 0.2525


0.252524733543396

샘플 prediction 출력

In [40]:
wide_window.plot(gain, plot_col='클로로필-a')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 학습데이터 테스트

In [ ]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
dim = np.prod(wide_window.dg.shape[1:]).astype(int)
print(dim)
n = (total_n//dim)*dim
print(n)
x = wide_window.dg.data[0:n].copy()
y = wide_window.dg.data[0:n].copy()
m = wide_window.dg.data_m[0:n]
x[m == 0] = np.nan
y[m == 1] = np.nan
print('x.shape =', x.shape)
x = x.reshape((-1,)+unit_shape)
y_true = y.reshape((-1,)+unit_shape)
print('x.shape =', x.shape)

In [ ]:
y_pred = gain.predict(x)

In [ ]:
y_pred = y_pred.reshape((n, 13))
x = x.reshape((n, 13))

In [ ]:
x.shape

In [ ]:
y_pred[~np.isnan(x)] = np.nan
plt.figure()
plt.plot(x[:, 8])
plt.plot(y_pred[:, 8])
plt.show()

## 원본 데이터 테스트

**원본데이터 shape**
```py
print(norm_df.shape)
(70068,13)
(rows, columns)
```
데이터 generator의 shape
```py
print(wide_window.dg.shape)
(128, 120, 13)
(batch_size, input_width, column)
```
```py
total_no = 70068
dim = 120*13
n = total_no//dim*dim
x.shape
(68640, 13)
x.reshape( (-1, 120, 13) )
```

In [41]:
norm_df = pd.concat(df,axis=0)

In [42]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

70128
(120, 11)
120
x.shape = (70080, 11)
x_reshape.shape = (584, 120, 11)
54114
y_true.shape= (70080, 11)
54114


(1, 120, 11)

In [43]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

19/19 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [44]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [45]:
print(total_n)
print(n)
print(total_n-n)

70128
70080
48


In [46]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(70080, 11) (120, 11)
(70128, 11)


그림용으로 nan 채우기

In [47]:
y_pred[~np.isnan(data)] = np.nan

In [48]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

70128
unit_shape= (120, 11)
120
n= 70080
(35040, 7)
(35040, 7)


In [50]:
# idx번째 데이터 출력
idx = 0
y_plt = gans[idx].copy()
y_plt[~np.isnan(oris[idx])] = np.nan
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(oris[idx][:, i])
    plt.plot(y_plt[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**self data 생성(가평)**

In [ ]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

**source-target data creation (target-가평, source-의암호)**

In [51]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


(35040, 11) (35040, 11)
(35040, 7) (35040, 7)
(35040, 18) (35040, 18)


1

In [52]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2]
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            
            x = np.append(x, [X], axis = 0)
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

In [53]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

**WaterDataGenerator 테스트**

In [54]:
OUT_STEPS = 24*5
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [55]:
it = iter(wdg)

In [56]:
x,y = next(it)
x.shape, y.shape

((128, 168, 18), (128, 120, 1))

**Water Dataset**

In [57]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.plot(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.plot(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [58]:
OUT_STEPS = 24*5
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

multi_window.plot2(plot_col=out_features[0])
multi_window

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total window size: 288
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [59]:
multi_window.plot2(plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
it = iter(multi_window.train)
x, y = next(it)
print(x.shape, y.shape)
#x, y = next(it#)

(128, 168, 18) (128, 120, 1)


**Baseline model**

In [61]:
multi_val_performance = {}
multi_performance = {}

In [62]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

#multi_val_performance = {}
#multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['Last'])
print('test performance = ', multi_performance['Last'])
multi_window.plot2(last_baseline, plot_col=out_features[0], plot_out_col=0)

100/100 [==============================] - 7s 74ms/step - loss: 0.3123 - mean_absolute_error: 0.3564
val performance = [0.3122413158416748, 0.356222003698349]
test performance =  [0.3411481976509094, 0.3654834032058716]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Water Data Generator test**

In [ ]:
wdg = WaterDataGenerator(
    train_df,
    batch_size = 32,
    input_width = 7,
    label_width = 3,
    shift = 3,
)

In [ ]:
it = iter(wdg)
x, y = next(it)
x.shape, y.shape

In [ ]:
last_baseline.evaluate(wdg)

# 모델 학습

In [63]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [64]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/400
1/1 [==============================] - 0s 439ms/step - loss: 0.7077 - mean_absolute_error: 0.6852 - val_loss: 0.7628 - val_mean_absolute_error: 0.6584
Epoch 2/400
1/1 [==============================] - 0s 196ms/step - loss: 0.5647 - mean_absolute_error: 0.6203 - val_loss: 0.8244 - val_mean_absolute_error: 0.6742
Epoch 3/400
1/1 [==============================] - 0s 221ms/step - loss: 0.7853 - mean_absolute_error: 0.6742 - val_loss: 0.9054 - val_mean_absolute_error: 0.7439
Epoch 4/400
1/1 [==============================] - 0s 220ms/step - loss: 0.7157 - mean_absolute_error: 0.6641 - val_loss: 0.8155 - val_mean_absolute_error: 0.6876
Epoch 5/400
1/1 [==============================] - 0s 253ms/step - loss: 0.7546 - mean_absolute_error: 0.6802 - val_loss: 0.8725 - val_mean_absolute_error: 0.6967
Epoch 6/400
1/1 [==============================] - 0s 227ms/step - loss: 0.6590 - mean_absolute_error: 0.6722 - val_loss: 0.8082 - val_mean_absolute_error: 0.6963
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 256ms/step - loss: 0.6541 - mean_absolute_error: 0.6094 - val_loss: 0.6642 - val_mean_absolute_error: 0.6206
Epoch 52/400
1/1 [==============================] - 0s 218ms/step - loss: 0.7204 - mean_absolute_error: 0.6019 - val_loss: 0.8143 - val_mean_absolute_error: 0.6503
Epoch 53/400
1/1 [==============================] - 0s 222ms/step - loss: 0.5451 - mean_absolute_error: 0.5765 - val_loss: 0.6707 - val_mean_absolute_error: 0.5895
Epoch 54/400
1/1 [==============================] - 0s 214ms/step - loss: 0.5869 - mean_absolute_error: 0.5724 - val_loss: 0.6772 - val_mean_absolute_error: 0.5909
Epoch 55/400
1/1 [==============================] - 0s 247ms/step - loss: 0.5448 - mean_absolute_error: 0.5544 - val_loss: 0.7635 - val_mean_absolute_error: 0.6317
Epoch 56/400
1/1 [==============================] - 0s 260ms/step - loss: 0.7375 - mean_absolute_error: 0.6098 - val_loss: 0.5368 - val_mean_absolute_error: 0.5484
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 260ms/step - loss: 0.3579 - mean_absolute_error: 0.4749 - val_loss: 0.8762 - val_mean_absolute_error: 0.6791
Epoch 102/400
1/1 [==============================] - 0s 248ms/step - loss: 0.4973 - mean_absolute_error: 0.5227 - val_loss: 0.7375 - val_mean_absolute_error: 0.6141
Epoch 103/400
1/1 [==============================] - 0s 262ms/step - loss: 0.5138 - mean_absolute_error: 0.5344 - val_loss: 0.7073 - val_mean_absolute_error: 0.5824
Epoch 104/400
1/1 [==============================] - 0s 217ms/step - loss: 0.6795 - mean_absolute_error: 0.5634 - val_loss: 0.5843 - val_mean_absolute_error: 0.5564
Epoch 105/400
1/1 [==============================] - 0s 218ms/step - loss: 0.3948 - mean_absolute_error: 0.5078 - val_loss: 0.7020 - val_mean_absolute_error: 0.5866
Epoch 106/400
1/1 [==============================] - 0s 253ms/step - loss: 0.5199 - mean_absolute_error: 0.5337 - val_loss: 0.6677 - val_mean_absolute_error: 0.5837
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 227ms/step - loss: 0.4311 - mean_absolute_error: 0.5047 - val_loss: 0.4591 - val_mean_absolute_error: 0.4698
Epoch 152/400
1/1 [==============================] - 0s 214ms/step - loss: 0.4297 - mean_absolute_error: 0.4827 - val_loss: 0.6449 - val_mean_absolute_error: 0.5700
Epoch 153/400
1/1 [==============================] - 0s 187ms/step - loss: 0.6324 - mean_absolute_error: 0.5268 - val_loss: 0.6027 - val_mean_absolute_error: 0.5481
Epoch 154/400
1/1 [==============================] - 0s 237ms/step - loss: 0.4818 - mean_absolute_error: 0.4892 - val_loss: 0.6192 - val_mean_absolute_error: 0.5503
Epoch 155/400
1/1 [==============================] - 0s 251ms/step - loss: 0.4533 - mean_absolute_error: 0.4906 - val_loss: 0.7353 - val_mean_absolute_error: 0.6039
Epoch 156/400
1/1 [==============================] - 0s 256ms/step - loss: 0.6023 - mean_absolute_error: 0.5427 - val_loss: 0.7116 - val_mean_absolute_error: 0.5852
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 247ms/step - loss: 0.4699 - mean_absolute_error: 0.4713 - val_loss: 0.5877 - val_mean_absolute_error: 0.5328
Epoch 202/400
1/1 [==============================] - 0s 216ms/step - loss: 0.4613 - mean_absolute_error: 0.4827 - val_loss: 0.6617 - val_mean_absolute_error: 0.5567
Epoch 203/400
1/1 [==============================] - 0s 237ms/step - loss: 0.3991 - mean_absolute_error: 0.4563 - val_loss: 0.6386 - val_mean_absolute_error: 0.5577
Epoch 204/400
1/1 [==============================] - 0s 249ms/step - loss: 0.4665 - mean_absolute_error: 0.4832 - val_loss: 0.6547 - val_mean_absolute_error: 0.5504
Epoch 205/400
1/1 [==============================] - 0s 217ms/step - loss: 0.3502 - mean_absolute_error: 0.4201 - val_loss: 0.5207 - val_mean_absolute_error: 0.5049
Epoch 206/400
1/1 [==============================] - 0s 244ms/step - loss: 0.3324 - mean_absolute_error: 0.4427 - val_loss: 0.5110 - val_mean_absolute_error: 0.4838
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 240ms/step - loss: 0.5487 - mean_absolute_error: 0.4812 - val_loss: 0.6339 - val_mean_absolute_error: 0.5622
Epoch 252/400
1/1 [==============================] - 0s 223ms/step - loss: 0.2800 - mean_absolute_error: 0.3801 - val_loss: 0.5192 - val_mean_absolute_error: 0.5047
Epoch 253/400
1/1 [==============================] - 0s 224ms/step - loss: 0.3255 - mean_absolute_error: 0.3840 - val_loss: 0.6021 - val_mean_absolute_error: 0.5426
Epoch 254/400
1/1 [==============================] - 0s 234ms/step - loss: 0.2826 - mean_absolute_error: 0.4021 - val_loss: 0.5475 - val_mean_absolute_error: 0.5146
Epoch 255/400
1/1 [==============================] - 0s 259ms/step - loss: 0.3315 - mean_absolute_error: 0.4391 - val_loss: 0.4765 - val_mean_absolute_error: 0.4796
Epoch 256/400
1/1 [==============================] - 0s 222ms/step - loss: 0.4362 - mean_absolute_error: 0.4408 - val_loss: 0.3811 - val_mean_absolute_error: 0.4414
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 228ms/step - loss: 0.3111 - mean_absolute_error: 0.4106 - val_loss: 0.3580 - val_mean_absolute_error: 0.4248
Epoch 302/400
1/1 [==============================] - 0s 231ms/step - loss: 0.3711 - mean_absolute_error: 0.4218 - val_loss: 0.4625 - val_mean_absolute_error: 0.4730
Epoch 303/400
1/1 [==============================] - 0s 233ms/step - loss: 0.4119 - mean_absolute_error: 0.4194 - val_loss: 0.5205 - val_mean_absolute_error: 0.5055
Epoch 304/400
1/1 [==============================] - 0s 220ms/step - loss: 0.2957 - mean_absolute_error: 0.3948 - val_loss: 0.5535 - val_mean_absolute_error: 0.5299
Epoch 305/400
1/1 [==============================] - 0s 211ms/step - loss: 0.5077 - mean_absolute_error: 0.4434 - val_loss: 0.4032 - val_mean_absolute_error: 0.4624
Epoch 306/400
1/1 [==============================] - 0s 199ms/step - loss: 0.3520 - mean_absolute_error: 0.4128 - val_loss: 0.4876 - val_mean_absolute_error: 0.4731
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 234ms/step - loss: 0.2862 - mean_absolute_error: 0.3867 - val_loss: 0.5183 - val_mean_absolute_error: 0.5089
Epoch 352/400
1/1 [==============================] - 0s 229ms/step - loss: 0.4613 - mean_absolute_error: 0.4494 - val_loss: 0.4192 - val_mean_absolute_error: 0.4521
Epoch 353/400
1/1 [==============================] - 0s 227ms/step - loss: 0.3060 - mean_absolute_error: 0.3866 - val_loss: 0.4179 - val_mean_absolute_error: 0.4540
Epoch 354/400
1/1 [==============================] - 0s 219ms/step - loss: 0.4095 - mean_absolute_error: 0.4420 - val_loss: 0.4717 - val_mean_absolute_error: 0.4950
Epoch 355/400
1/1 [==============================] - 0s 218ms/step - loss: 0.3237 - mean_absolute_error: 0.3874 - val_loss: 0.4768 - val_mean_absolute_error: 0.4889
Epoch 356/400
1/1 [==============================] - 0s 250ms/step - loss: 0.2912 - mean_absolute_error: 0.3895 - val_loss: 0.4696 - val_mean_absolute_error: 0.4723
Epoch 357/

100/100 [==============================] - 8s 76ms/step - loss: 0.4605 - mean_absolute_error: 0.4784
val performance = [0.46052706241607666, 0.47844693064689636]
test performance =  [0.3657521903514862, 0.41969433426856995]


In [65]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [66]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**muti_step dense**

In [68]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Flatten(),
    # Shape => [batch, 1, dense_units]
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [69]:
history = compile_and_fit(multi_dense_model, multi_window)
multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['Dense'])
print('test performance = ', multi_performance['Dense'])

Epoch 1/400
1/1 [==============================] - 0s 462ms/step - loss: 1.0076 - mean_absolute_error: 0.7510 - val_loss: 0.8681 - val_mean_absolute_error: 0.7054
Epoch 2/400
1/1 [==============================] - 0s 245ms/step - loss: 0.8596 - mean_absolute_error: 0.7286 - val_loss: 0.8141 - val_mean_absolute_error: 0.6727
Epoch 3/400
1/1 [==============================] - 0s 219ms/step - loss: 0.8927 - mean_absolute_error: 0.7004 - val_loss: 0.8552 - val_mean_absolute_error: 0.6755
Epoch 4/400
1/1 [==============================] - 0s 205ms/step - loss: 0.8445 - mean_absolute_error: 0.6641 - val_loss: 0.7977 - val_mean_absolute_error: 0.6612
Epoch 5/400
1/1 [==============================] - 0s 205ms/step - loss: 0.7174 - mean_absolute_error: 0.5856 - val_loss: 0.8047 - val_mean_absolute_error: 0.6350
Epoch 6/400
1/1 [==============================] - 0s 229ms/step - loss: 0.4779 - mean_absolute_error: 0.5275 - val_loss: 0.8743 - val_mean_absolute_error: 0.6568
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 245ms/step - loss: 0.3666 - mean_absolute_error: 0.3856 - val_loss: 0.2602 - val_mean_absolute_error: 0.3595
Epoch 52/400
1/1 [==============================] - 0s 260ms/step - loss: 0.2742 - mean_absolute_error: 0.3566 - val_loss: 0.3703 - val_mean_absolute_error: 0.4475
Epoch 53/400
1/1 [==============================] - 0s 253ms/step - loss: 0.2207 - mean_absolute_error: 0.3223 - val_loss: 0.2643 - val_mean_absolute_error: 0.3765
Epoch 54/400
1/1 [==============================] - 0s 219ms/step - loss: 0.2347 - mean_absolute_error: 0.3547 - val_loss: 0.3449 - val_mean_absolute_error: 0.4334
Epoch 55/400
1/1 [==============================] - 0s 238ms/step - loss: 0.1771 - mean_absolute_error: 0.2978 - val_loss: 0.2502 - val_mean_absolute_error: 0.3501
Epoch 56/400
1/1 [==============================] - 0s 205ms/step - loss: 0.2764 - mean_absolute_error: 0.3444 - val_loss: 0.2795 - val_mean_absolute_error: 0.3617
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 245ms/step - loss: 0.2184 - mean_absolute_error: 0.3269 - val_loss: 0.2430 - val_mean_absolute_error: 0.3361
Epoch 102/400
1/1 [==============================] - 0s 227ms/step - loss: 0.2241 - mean_absolute_error: 0.3151 - val_loss: 0.2475 - val_mean_absolute_error: 0.3445
Epoch 103/400
1/1 [==============================] - 0s 249ms/step - loss: 0.2075 - mean_absolute_error: 0.3063 - val_loss: 0.2887 - val_mean_absolute_error: 0.3643
Epoch 104/400
1/1 [==============================] - 0s 231ms/step - loss: 0.2282 - mean_absolute_error: 0.3309 - val_loss: 0.3295 - val_mean_absolute_error: 0.3836
Epoch 105/400
1/1 [==============================] - 0s 255ms/step - loss: 0.2709 - mean_absolute_error: 0.3619 - val_loss: 0.2531 - val_mean_absolute_error: 0.3332
Epoch 106/400
1/1 [==============================] - 0s 254ms/step - loss: 0.2623 - mean_absolute_error: 0.3160 - val_loss: 0.2477 - val_mean_absolute_error: 0.3483
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 264ms/step - loss: 0.1986 - mean_absolute_error: 0.3214 - val_loss: 0.2020 - val_mean_absolute_error: 0.3167
Epoch 152/400
1/1 [==============================] - 0s 217ms/step - loss: 0.2244 - mean_absolute_error: 0.3226 - val_loss: 0.2755 - val_mean_absolute_error: 0.3637
Epoch 153/400
1/1 [==============================] - 0s 195ms/step - loss: 0.2722 - mean_absolute_error: 0.3588 - val_loss: 0.3109 - val_mean_absolute_error: 0.3845
Epoch 154/400
1/1 [==============================] - 0s 226ms/step - loss: 0.2352 - mean_absolute_error: 0.3208 - val_loss: 0.2886 - val_mean_absolute_error: 0.3724
Epoch 155/400
1/1 [==============================] - 0s 208ms/step - loss: 0.2539 - mean_absolute_error: 0.3581 - val_loss: 0.3278 - val_mean_absolute_error: 0.3997
Epoch 156/400
1/1 [==============================] - 0s 266ms/step - loss: 0.2566 - mean_absolute_error: 0.3164 - val_loss: 0.2833 - val_mean_absolute_error: 0.3452
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 244ms/step - loss: 0.2240 - mean_absolute_error: 0.3131 - val_loss: 0.2638 - val_mean_absolute_error: 0.3548
Epoch 202/400
1/1 [==============================] - 0s 222ms/step - loss: 0.1888 - mean_absolute_error: 0.2994 - val_loss: 0.2359 - val_mean_absolute_error: 0.3370
Epoch 203/400
1/1 [==============================] - 0s 270ms/step - loss: 0.1699 - mean_absolute_error: 0.2768 - val_loss: 0.2760 - val_mean_absolute_error: 0.3511
Epoch 204/400
1/1 [==============================] - 0s 345ms/step - loss: 0.2078 - mean_absolute_error: 0.2997 - val_loss: 0.2462 - val_mean_absolute_error: 0.3509
Epoch 205/400
1/1 [==============================] - 0s 210ms/step - loss: 0.2338 - mean_absolute_error: 0.3205 - val_loss: 0.3557 - val_mean_absolute_error: 0.3980
Epoch 206/400
1/1 [==============================] - 0s 223ms/step - loss: 0.2020 - mean_absolute_error: 0.3141 - val_loss: 0.3101 - val_mean_absolute_error: 0.3835
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 252ms/step - loss: 0.1747 - mean_absolute_error: 0.2836 - val_loss: 0.3677 - val_mean_absolute_error: 0.4134
Epoch 252/400
1/1 [==============================] - 0s 228ms/step - loss: 0.2139 - mean_absolute_error: 0.3019 - val_loss: 0.4670 - val_mean_absolute_error: 0.4663
Epoch 253/400
1/1 [==============================] - 0s 235ms/step - loss: 0.2009 - mean_absolute_error: 0.3149 - val_loss: 0.4194 - val_mean_absolute_error: 0.4335
Epoch 254/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1736 - mean_absolute_error: 0.2870 - val_loss: 0.3482 - val_mean_absolute_error: 0.4059
Epoch 255/400
1/1 [==============================] - 0s 265ms/step - loss: 0.1451 - mean_absolute_error: 0.2730 - val_loss: 0.3903 - val_mean_absolute_error: 0.4231
Epoch 256/400
1/1 [==============================] - 0s 232ms/step - loss: 0.2107 - mean_absolute_error: 0.3014 - val_loss: 0.3596 - val_mean_absolute_error: 0.4052
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 192ms/step - loss: 0.1815 - mean_absolute_error: 0.2914 - val_loss: 0.3608 - val_mean_absolute_error: 0.4120
Epoch 302/400
1/1 [==============================] - 0s 224ms/step - loss: 0.1762 - mean_absolute_error: 0.2952 - val_loss: 0.4278 - val_mean_absolute_error: 0.4339
Epoch 303/400
1/1 [==============================] - 0s 273ms/step - loss: 0.2249 - mean_absolute_error: 0.3267 - val_loss: 0.3670 - val_mean_absolute_error: 0.4069
Epoch 304/400
1/1 [==============================] - 0s 279ms/step - loss: 0.1746 - mean_absolute_error: 0.2935 - val_loss: 0.4684 - val_mean_absolute_error: 0.4564
Epoch 305/400
1/1 [==============================] - 0s 213ms/step - loss: 0.1795 - mean_absolute_error: 0.2972 - val_loss: 0.4418 - val_mean_absolute_error: 0.4479
Epoch 306/400
1/1 [==============================] - 0s 241ms/step - loss: 0.1863 - mean_absolute_error: 0.2961 - val_loss: 0.4633 - val_mean_absolute_error: 0.4545
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 222ms/step - loss: 0.1769 - mean_absolute_error: 0.2975 - val_loss: 0.4483 - val_mean_absolute_error: 0.4733
Epoch 352/400
1/1 [==============================] - 0s 193ms/step - loss: 0.1737 - mean_absolute_error: 0.2974 - val_loss: 0.4041 - val_mean_absolute_error: 0.4339
Epoch 353/400
1/1 [==============================] - 0s 243ms/step - loss: 0.1871 - mean_absolute_error: 0.2788 - val_loss: 0.3841 - val_mean_absolute_error: 0.4207
Epoch 354/400
1/1 [==============================] - 0s 202ms/step - loss: 0.1922 - mean_absolute_error: 0.2997 - val_loss: 0.4367 - val_mean_absolute_error: 0.4552
Epoch 355/400
1/1 [==============================] - 0s 211ms/step - loss: 0.1768 - mean_absolute_error: 0.2923 - val_loss: 0.4126 - val_mean_absolute_error: 0.4366
Epoch 356/400
1/1 [==============================] - 0s 196ms/step - loss: 0.2225 - mean_absolute_error: 0.3119 - val_loss: 0.4764 - val_mean_absolute_error: 0.4767
Epoch 357/

100/100 [==============================] - 8s 76ms/step - loss: 0.2230 - mean_absolute_error: 0.3190
val performance = [0.3648418188095093, 0.4174003005027771]
test performance =  [0.22299309074878693, 0.31901800632476807]


In [70]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
multi_window.plot2(multi_dense_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Conv model**

In [72]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
CONV_WIDTH = 3
CONV_LAYER_NO = 6
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [ ]:
CONV_WIDTH = 11
CONV_LAYER_NO = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [ ]:
CONV_WIDTH = 11
#CONV_LAYER_NO = 3
multi_conv_model = tf.keras.Sequential([
    keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(512, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.GlobalAveragePooling1D(),
    #keras.layers.Dropout(0.5),
    
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [ ]:
CONV_WIDTH = 11
#CONV_LAYER_NO = 3
multi_conv_model = tf.keras.Sequential([
    keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Conv1D(256, strides=2, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Conv1D(256, strides=2, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Conv1D(512, strides=2, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Conv1D(1024, strides=2, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.GlobalAveragePooling1D(),
    #keras.layers.Dropout(0.5),
    
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [73]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test.repeat(-1), steps=100, verbose=1)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 0.6137 - mean_absolute_error: 0.6230 - val_loss: 1.1786 - val_mean_absolute_error: 0.8209
Epoch 2/400
1/1 [==============================] - 0s 237ms/step - loss: 0.6915 - mean_absolute_error: 0.6588 - val_loss: 0.9750 - val_mean_absolute_error: 0.7405
Epoch 3/400
1/1 [==============================] - 0s 204ms/step - loss: 0.8119 - mean_absolute_error: 0.6457 - val_loss: 0.7558 - val_mean_absolute_error: 0.6071
Epoch 4/400
1/1 [==============================] - 0s 209ms/step - loss: 0.7199 - mean_absolute_error: 0.5813 - val_loss: 0.8502 - val_mean_absolute_error: 0.6479
Epoch 5/400
1/1 [==============================] - 0s 253ms/step - loss: 0.5458 - mean_absolute_error: 0.5209 - val_loss: 0.6866 - val_mean_absolute_error: 0.5471
Epoch 6/400
1/1 [==============================] - 0s 242ms/step - loss: 0.5126 - mean_absolute_error: 0.4668 - val_loss: 0.7895 - val_mean_absolute_error: 0.5843
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 232ms/step - loss: 0.3026 - mean_absolute_error: 0.3669 - val_loss: 0.2942 - val_mean_absolute_error: 0.3816
Epoch 52/400
1/1 [==============================] - 0s 232ms/step - loss: 0.1781 - mean_absolute_error: 0.3033 - val_loss: 0.3468 - val_mean_absolute_error: 0.4157
Epoch 53/400
1/1 [==============================] - 0s 222ms/step - loss: 0.2006 - mean_absolute_error: 0.3224 - val_loss: 0.2782 - val_mean_absolute_error: 0.3717
Epoch 54/400
1/1 [==============================] - 0s 203ms/step - loss: 0.2184 - mean_absolute_error: 0.3068 - val_loss: 0.2206 - val_mean_absolute_error: 0.3413
Epoch 55/400
1/1 [==============================] - 0s 223ms/step - loss: 0.2077 - mean_absolute_error: 0.3205 - val_loss: 0.3277 - val_mean_absolute_error: 0.4045
Epoch 56/400
1/1 [==============================] - 0s 223ms/step - loss: 0.2109 - mean_absolute_error: 0.3126 - val_loss: 0.2923 - val_mean_absolute_error: 0.3858
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 234ms/step - loss: 0.2018 - mean_absolute_error: 0.3250 - val_loss: 0.2380 - val_mean_absolute_error: 0.3364
Epoch 102/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1939 - mean_absolute_error: 0.2979 - val_loss: 0.4281 - val_mean_absolute_error: 0.4347
Epoch 103/400
1/1 [==============================] - 0s 211ms/step - loss: 0.2142 - mean_absolute_error: 0.3118 - val_loss: 0.2675 - val_mean_absolute_error: 0.3636
Epoch 104/400
1/1 [==============================] - 0s 237ms/step - loss: 0.2177 - mean_absolute_error: 0.3395 - val_loss: 0.4249 - val_mean_absolute_error: 0.4188
Epoch 105/400
1/1 [==============================] - 0s 213ms/step - loss: 0.1264 - mean_absolute_error: 0.2771 - val_loss: 0.3171 - val_mean_absolute_error: 0.3655
Epoch 106/400
1/1 [==============================] - 0s 239ms/step - loss: 0.1873 - mean_absolute_error: 0.3137 - val_loss: 0.3067 - val_mean_absolute_error: 0.3881
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 229ms/step - loss: 0.1706 - mean_absolute_error: 0.3033 - val_loss: 0.3651 - val_mean_absolute_error: 0.4106
Epoch 152/400
1/1 [==============================] - 0s 225ms/step - loss: 0.1931 - mean_absolute_error: 0.3179 - val_loss: 0.3250 - val_mean_absolute_error: 0.3950
Epoch 153/400
1/1 [==============================] - 0s 232ms/step - loss: 0.1717 - mean_absolute_error: 0.2985 - val_loss: 0.4380 - val_mean_absolute_error: 0.4432
Epoch 154/400
1/1 [==============================] - 0s 242ms/step - loss: 0.1423 - mean_absolute_error: 0.2653 - val_loss: 0.5349 - val_mean_absolute_error: 0.4724
Epoch 155/400
1/1 [==============================] - 0s 251ms/step - loss: 0.2403 - mean_absolute_error: 0.3268 - val_loss: 0.4678 - val_mean_absolute_error: 0.4634
Epoch 156/400
1/1 [==============================] - 0s 281ms/step - loss: 0.2019 - mean_absolute_error: 0.3112 - val_loss: 0.3951 - val_mean_absolute_error: 0.4186
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 238ms/step - loss: 0.1353 - mean_absolute_error: 0.2688 - val_loss: 0.5320 - val_mean_absolute_error: 0.5127
Epoch 202/400
1/1 [==============================] - 0s 219ms/step - loss: 0.1782 - mean_absolute_error: 0.2924 - val_loss: 0.5098 - val_mean_absolute_error: 0.4837
Epoch 203/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1992 - mean_absolute_error: 0.3232 - val_loss: 0.5011 - val_mean_absolute_error: 0.4697
Epoch 204/400
1/1 [==============================] - 0s 194ms/step - loss: 0.1749 - mean_absolute_error: 0.2936 - val_loss: 0.4958 - val_mean_absolute_error: 0.4779
Epoch 205/400
1/1 [==============================] - 0s 213ms/step - loss: 0.1613 - mean_absolute_error: 0.2827 - val_loss: 0.4221 - val_mean_absolute_error: 0.4337
Epoch 206/400
1/1 [==============================] - 0s 239ms/step - loss: 0.1495 - mean_absolute_error: 0.2715 - val_loss: 0.4194 - val_mean_absolute_error: 0.4398
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 271ms/step - loss: 0.1107 - mean_absolute_error: 0.2371 - val_loss: 0.6048 - val_mean_absolute_error: 0.5075
Epoch 252/400
1/1 [==============================] - 0s 225ms/step - loss: 0.1810 - mean_absolute_error: 0.2784 - val_loss: 0.3739 - val_mean_absolute_error: 0.4032
Epoch 253/400
1/1 [==============================] - 0s 221ms/step - loss: 0.1505 - mean_absolute_error: 0.2697 - val_loss: 0.4009 - val_mean_absolute_error: 0.4304
Epoch 254/400
1/1 [==============================] - 0s 214ms/step - loss: 0.1699 - mean_absolute_error: 0.2874 - val_loss: 0.4400 - val_mean_absolute_error: 0.4762
Epoch 255/400
1/1 [==============================] - 0s 212ms/step - loss: 0.1425 - mean_absolute_error: 0.2692 - val_loss: 0.5261 - val_mean_absolute_error: 0.4847
Epoch 256/400
1/1 [==============================] - 0s 252ms/step - loss: 0.1766 - mean_absolute_error: 0.2881 - val_loss: 0.4233 - val_mean_absolute_error: 0.4715
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 227ms/step - loss: 0.1324 - mean_absolute_error: 0.2608 - val_loss: 0.3374 - val_mean_absolute_error: 0.4133
Epoch 302/400
1/1 [==============================] - 0s 264ms/step - loss: 0.1658 - mean_absolute_error: 0.2921 - val_loss: 0.5022 - val_mean_absolute_error: 0.5014
Epoch 303/400
1/1 [==============================] - 0s 248ms/step - loss: 0.1313 - mean_absolute_error: 0.2590 - val_loss: 0.5328 - val_mean_absolute_error: 0.4905
Epoch 304/400
1/1 [==============================] - 0s 191ms/step - loss: 0.1486 - mean_absolute_error: 0.2742 - val_loss: 0.4786 - val_mean_absolute_error: 0.4732
Epoch 305/400
1/1 [==============================] - 0s 216ms/step - loss: 0.1439 - mean_absolute_error: 0.2587 - val_loss: 0.4567 - val_mean_absolute_error: 0.4647
Epoch 306/400
1/1 [==============================] - 0s 242ms/step - loss: 0.1473 - mean_absolute_error: 0.2741 - val_loss: 0.4467 - val_mean_absolute_error: 0.4757
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 231ms/step - loss: 0.1307 - mean_absolute_error: 0.2579 - val_loss: 0.4887 - val_mean_absolute_error: 0.4837
Epoch 352/400
1/1 [==============================] - 0s 232ms/step - loss: 0.1909 - mean_absolute_error: 0.2929 - val_loss: 0.4714 - val_mean_absolute_error: 0.4742
Epoch 353/400
1/1 [==============================] - 0s 254ms/step - loss: 0.1205 - mean_absolute_error: 0.2475 - val_loss: 0.3963 - val_mean_absolute_error: 0.4537
Epoch 354/400
1/1 [==============================] - 0s 214ms/step - loss: 0.1702 - mean_absolute_error: 0.2787 - val_loss: 0.5001 - val_mean_absolute_error: 0.4688
Epoch 355/400
1/1 [==============================] - 0s 243ms/step - loss: 0.1765 - mean_absolute_error: 0.2803 - val_loss: 0.4063 - val_mean_absolute_error: 0.4521
Epoch 356/400
1/1 [==============================] - 0s 206ms/step - loss: 0.1621 - mean_absolute_error: 0.2826 - val_loss: 0.5227 - val_mean_absolute_error: 0.5163
Epoch 357/

100/100 [==============================] - 7s 72ms/step - loss: 0.2355 - mean_absolute_error: 0.3187
val performance = [0.4801466763019562, 0.4762771725654602]
test performance =  [0.23554949462413788, 0.3187253177165985]


In [74]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**RNN(Elman)**

In [76]:
multi_elman_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_elman_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Elman'] = multi_elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Elman'] = multi_elman_model.evaluate(multi_window.test.repeat(-1), steps=100, verbose=1)
print('val performance =', multi_val_performance['Elman'])
print('test performance = ', multi_performance['Elman'])

Epoch 1/400
1/1 [==============================] - 1s 774ms/step - loss: 0.8073 - mean_absolute_error: 0.6711 - val_loss: 1.1355 - val_mean_absolute_error: 0.8385
Epoch 2/400
1/1 [==============================] - 0s 284ms/step - loss: 0.6390 - mean_absolute_error: 0.6401 - val_loss: 1.0013 - val_mean_absolute_error: 0.7507
Epoch 3/400
1/1 [==============================] - 0s 259ms/step - loss: 0.8419 - mean_absolute_error: 0.6824 - val_loss: 0.8563 - val_mean_absolute_error: 0.6647
Epoch 4/400
1/1 [==============================] - 0s 217ms/step - loss: 0.5850 - mean_absolute_error: 0.6255 - val_loss: 0.6364 - val_mean_absolute_error: 0.5935
Epoch 5/400
1/1 [==============================] - 0s 248ms/step - loss: 0.5298 - mean_absolute_error: 0.5655 - val_loss: 0.7543 - val_mean_absolute_error: 0.6243
Epoch 6/400
1/1 [==============================] - 0s 270ms/step - loss: 0.8524 - mean_absolute_error: 0.6334 - val_loss: 0.5144 - val_mean_absolute_error: 0.5189
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 257ms/step - loss: 0.3503 - mean_absolute_error: 0.3744 - val_loss: 0.3472 - val_mean_absolute_error: 0.4176
Epoch 52/400
1/1 [==============================] - 0s 235ms/step - loss: 0.3091 - mean_absolute_error: 0.3560 - val_loss: 0.3426 - val_mean_absolute_error: 0.4119
Epoch 53/400
1/1 [==============================] - 0s 249ms/step - loss: 0.3448 - mean_absolute_error: 0.3592 - val_loss: 0.3345 - val_mean_absolute_error: 0.4098
Epoch 54/400
1/1 [==============================] - 0s 287ms/step - loss: 0.2202 - mean_absolute_error: 0.3152 - val_loss: 0.2819 - val_mean_absolute_error: 0.3944
Epoch 55/400
1/1 [==============================] - 0s 265ms/step - loss: 0.3109 - mean_absolute_error: 0.3700 - val_loss: 0.3493 - val_mean_absolute_error: 0.4118
Epoch 56/400
1/1 [==============================] - 0s 302ms/step - loss: 0.3541 - mean_absolute_error: 0.4049 - val_loss: 0.2685 - val_mean_absolute_error: 0.3744
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 258ms/step - loss: 0.2408 - mean_absolute_error: 0.3219 - val_loss: 0.3081 - val_mean_absolute_error: 0.4190
Epoch 102/400
1/1 [==============================] - 0s 260ms/step - loss: 0.2423 - mean_absolute_error: 0.3258 - val_loss: 0.2743 - val_mean_absolute_error: 0.3998
Epoch 103/400
1/1 [==============================] - 0s 260ms/step - loss: 0.1982 - mean_absolute_error: 0.3200 - val_loss: 0.3220 - val_mean_absolute_error: 0.4127
Epoch 104/400
1/1 [==============================] - 0s 268ms/step - loss: 0.2432 - mean_absolute_error: 0.3404 - val_loss: 0.2695 - val_mean_absolute_error: 0.3841
Epoch 105/400
1/1 [==============================] - 0s 282ms/step - loss: 0.2630 - mean_absolute_error: 0.3352 - val_loss: 0.2640 - val_mean_absolute_error: 0.3841
Epoch 106/400
1/1 [==============================] - 0s 230ms/step - loss: 0.1900 - mean_absolute_error: 0.3055 - val_loss: 0.3033 - val_mean_absolute_error: 0.3991
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 263ms/step - loss: 0.2127 - mean_absolute_error: 0.3090 - val_loss: 0.3244 - val_mean_absolute_error: 0.4177
Epoch 152/400
1/1 [==============================] - 0s 228ms/step - loss: 0.1895 - mean_absolute_error: 0.3107 - val_loss: 0.2918 - val_mean_absolute_error: 0.3764
Epoch 153/400
1/1 [==============================] - 0s 298ms/step - loss: 0.2287 - mean_absolute_error: 0.3455 - val_loss: 0.4692 - val_mean_absolute_error: 0.4500
Epoch 154/400
1/1 [==============================] - 0s 291ms/step - loss: 0.2615 - mean_absolute_error: 0.3451 - val_loss: 0.4164 - val_mean_absolute_error: 0.4655
Epoch 155/400
1/1 [==============================] - 0s 222ms/step - loss: 0.3330 - mean_absolute_error: 0.3717 - val_loss: 0.3824 - val_mean_absolute_error: 0.4353
Epoch 156/400
1/1 [==============================] - 0s 266ms/step - loss: 0.2859 - mean_absolute_error: 0.3508 - val_loss: 0.4128 - val_mean_absolute_error: 0.4717
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 245ms/step - loss: 0.2242 - mean_absolute_error: 0.3283 - val_loss: 0.2916 - val_mean_absolute_error: 0.3814
Epoch 202/400
1/1 [==============================] - 0s 247ms/step - loss: 0.1929 - mean_absolute_error: 0.3069 - val_loss: 0.2949 - val_mean_absolute_error: 0.3720
Epoch 203/400
1/1 [==============================] - 0s 260ms/step - loss: 0.2209 - mean_absolute_error: 0.3166 - val_loss: 0.3521 - val_mean_absolute_error: 0.4331
Epoch 204/400
1/1 [==============================] - 0s 262ms/step - loss: 0.2548 - mean_absolute_error: 0.3337 - val_loss: 0.3493 - val_mean_absolute_error: 0.4339
Epoch 205/400
1/1 [==============================] - 0s 258ms/step - loss: 0.1472 - mean_absolute_error: 0.2723 - val_loss: 0.4905 - val_mean_absolute_error: 0.5076
Epoch 206/400
1/1 [==============================] - 0s 260ms/step - loss: 0.1953 - mean_absolute_error: 0.2988 - val_loss: 0.5724 - val_mean_absolute_error: 0.5450
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 271ms/step - loss: 0.1715 - mean_absolute_error: 0.2848 - val_loss: 0.3784 - val_mean_absolute_error: 0.4381
Epoch 252/400
1/1 [==============================] - 0s 256ms/step - loss: 0.1930 - mean_absolute_error: 0.3062 - val_loss: 0.3784 - val_mean_absolute_error: 0.4666
Epoch 253/400
1/1 [==============================] - 0s 291ms/step - loss: 0.1670 - mean_absolute_error: 0.2941 - val_loss: 0.3973 - val_mean_absolute_error: 0.4532
Epoch 254/400
1/1 [==============================] - 0s 254ms/step - loss: 0.1329 - mean_absolute_error: 0.2608 - val_loss: 0.4368 - val_mean_absolute_error: 0.4700
Epoch 255/400
1/1 [==============================] - 0s 255ms/step - loss: 0.1406 - mean_absolute_error: 0.2708 - val_loss: 0.4076 - val_mean_absolute_error: 0.4528
Epoch 256/400
1/1 [==============================] - 0s 234ms/step - loss: 0.1681 - mean_absolute_error: 0.2942 - val_loss: 0.4675 - val_mean_absolute_error: 0.4661
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 236ms/step - loss: 0.1773 - mean_absolute_error: 0.2675 - val_loss: 0.5926 - val_mean_absolute_error: 0.5665
Epoch 302/400
1/1 [==============================] - 0s 299ms/step - loss: 0.1729 - mean_absolute_error: 0.2870 - val_loss: 0.4892 - val_mean_absolute_error: 0.4974
Epoch 303/400
1/1 [==============================] - 0s 252ms/step - loss: 0.1780 - mean_absolute_error: 0.2795 - val_loss: 0.4107 - val_mean_absolute_error: 0.4519
Epoch 304/400
1/1 [==============================] - 0s 239ms/step - loss: 0.1849 - mean_absolute_error: 0.2769 - val_loss: 0.4530 - val_mean_absolute_error: 0.4756
Epoch 305/400
1/1 [==============================] - 0s 258ms/step - loss: 0.1331 - mean_absolute_error: 0.2502 - val_loss: 0.3549 - val_mean_absolute_error: 0.4319
Epoch 306/400
1/1 [==============================] - 0s 246ms/step - loss: 0.1512 - mean_absolute_error: 0.2656 - val_loss: 0.3136 - val_mean_absolute_error: 0.4054
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 243ms/step - loss: 0.1515 - mean_absolute_error: 0.2620 - val_loss: 0.4681 - val_mean_absolute_error: 0.4848
Epoch 352/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1444 - mean_absolute_error: 0.2533 - val_loss: 0.4778 - val_mean_absolute_error: 0.4715
Epoch 353/400
1/1 [==============================] - 0s 257ms/step - loss: 0.1134 - mean_absolute_error: 0.2392 - val_loss: 0.5847 - val_mean_absolute_error: 0.5376
Epoch 354/400
1/1 [==============================] - 0s 260ms/step - loss: 0.1342 - mean_absolute_error: 0.2667 - val_loss: 0.4565 - val_mean_absolute_error: 0.4504
Epoch 355/400
1/1 [==============================] - 0s 219ms/step - loss: 0.1458 - mean_absolute_error: 0.2689 - val_loss: 0.5294 - val_mean_absolute_error: 0.5193
Epoch 356/400
1/1 [==============================] - 0s 241ms/step - loss: 0.1290 - mean_absolute_error: 0.2520 - val_loss: 0.3761 - val_mean_absolute_error: 0.4334
Epoch 357/

100/100 [==============================] - 8s 79ms/step - loss: 0.2653 - mean_absolute_error: 0.3449
val performance = [0.5694880485534668, 0.5502445697784424]
test performance =  [0.2653120756149292, 0.34490901231765747]


In [77]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
multi_window.plot2(multi_elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**RNN(GRU)**

In [79]:
MAX_EPOCHS = 150
multi_gru_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.GRU(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_gru_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.test.repeat(-1), steps=100, verbose=1)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 863ms/step - loss: 0.8332 - mean_absolute_error: 0.7081 - val_loss: 0.9237 - val_mean_absolute_error: 0.7370
Epoch 2/150
1/1 [==============================] - 0s 280ms/step - loss: 0.6640 - mean_absolute_error: 0.6114 - val_loss: 0.9185 - val_mean_absolute_error: 0.7308
Epoch 3/150
1/1 [==============================] - 0s 285ms/step - loss: 0.8355 - mean_absolute_error: 0.6861 - val_loss: 0.9838 - val_mean_absolute_error: 0.7526
Epoch 4/150
1/1 [==============================] - 0s 252ms/step - loss: 0.7715 - mean_absolute_error: 0.6889 - val_loss: 1.0089 - val_mean_absolute_error: 0.7888
Epoch 5/150
1/1 [==============================] - 0s 276ms/step - loss: 0.8235 - mean_absolute_error: 0.6932 - val_loss: 0.8023 - val_mean_absolute_error: 0.6397
Epoch 6/150
1/1 [==============================] - 0s 316ms/step - loss: 0.7230 - mean_absolute_error: 0.6478 - val_loss: 0.6707 - val_mean_absolute_error: 0.6049
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 306ms/step - loss: 0.3005 - mean_absolute_error: 0.3963 - val_loss: 0.3041 - val_mean_absolute_error: 0.4240
Epoch 52/150
1/1 [==============================] - 0s 267ms/step - loss: 0.2779 - mean_absolute_error: 0.3656 - val_loss: 0.3165 - val_mean_absolute_error: 0.4115
Epoch 53/150
1/1 [==============================] - 0s 279ms/step - loss: 0.2389 - mean_absolute_error: 0.3423 - val_loss: 0.3115 - val_mean_absolute_error: 0.3955
Epoch 54/150
1/1 [==============================] - 0s 262ms/step - loss: 0.2870 - mean_absolute_error: 0.3747 - val_loss: 0.2145 - val_mean_absolute_error: 0.3422
Epoch 55/150
1/1 [==============================] - 0s 282ms/step - loss: 0.2875 - mean_absolute_error: 0.3800 - val_loss: 0.3029 - val_mean_absolute_error: 0.4042
Epoch 56/150
1/1 [==============================] - 0s 260ms/step - loss: 0.2645 - mean_absolute_error: 0.3432 - val_loss: 0.2882 - val_mean_absolute_error: 0.3928
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 321ms/step - loss: 0.2240 - mean_absolute_error: 0.3345 - val_loss: 0.2983 - val_mean_absolute_error: 0.3785
Epoch 102/150
1/1 [==============================] - 0s 312ms/step - loss: 0.1552 - mean_absolute_error: 0.2961 - val_loss: 0.2365 - val_mean_absolute_error: 0.3502
Epoch 103/150
1/1 [==============================] - 0s 309ms/step - loss: 0.2737 - mean_absolute_error: 0.3424 - val_loss: 0.2568 - val_mean_absolute_error: 0.3445
Epoch 104/150
1/1 [==============================] - 0s 288ms/step - loss: 0.2178 - mean_absolute_error: 0.3365 - val_loss: 0.2176 - val_mean_absolute_error: 0.3251
Epoch 105/150
1/1 [==============================] - 0s 322ms/step - loss: 0.2792 - mean_absolute_error: 0.3395 - val_loss: 0.2358 - val_mean_absolute_error: 0.3396
Epoch 106/150
1/1 [==============================] - 0s 303ms/step - loss: 0.2975 - mean_absolute_error: 0.3603 - val_loss: 0.2268 - val_mean_absolute_error: 0.3469
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 91ms/step - loss: 0.2464 - mean_absolute_error: 0.3398
val performance = [0.29944345355033875, 0.37468481063842773]
test performance =  [0.2463562935590744, 0.33977821469306946]


In [80]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
multi_window.plot2(multi_gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**RNN(lstm)**

In [82]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(256, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test.repeat(-1), steps=100, verbose=1)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.7543 - mean_absolute_error: 0.6905 - val_loss: 0.8227 - val_mean_absolute_error: 0.6753
Epoch 2/150
1/1 [==============================] - 0s 281ms/step - loss: 0.8610 - mean_absolute_error: 0.7310 - val_loss: 0.8272 - val_mean_absolute_error: 0.6965
Epoch 3/150
1/1 [==============================] - 0s 265ms/step - loss: 0.6790 - mean_absolute_error: 0.6642 - val_loss: 0.8766 - val_mean_absolute_error: 0.7144
Epoch 4/150
1/1 [==============================] - 0s 262ms/step - loss: 0.7197 - mean_absolute_error: 0.6313 - val_loss: 0.6280 - val_mean_absolute_error: 0.5942
Epoch 5/150
1/1 [==============================] - 0s 240ms/step - loss: 0.8135 - mean_absolute_error: 0.6692 - val_loss: 0.5917 - val_mean_absolute_error: 0.5594
Epoch 6/150
1/1 [==============================] - 0s 221ms/step - loss: 0.5130 - mean_absolute_error: 0.5690 - val_loss: 0.4451 - val_mean_absolute_error: 0.4755
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 238ms/step - loss: 0.1888 - mean_absolute_error: 0.3085 - val_loss: 0.3221 - val_mean_absolute_error: 0.4216
Epoch 52/150
1/1 [==============================] - 0s 249ms/step - loss: 0.2448 - mean_absolute_error: 0.3356 - val_loss: 0.3425 - val_mean_absolute_error: 0.4369
Epoch 53/150
1/1 [==============================] - 0s 266ms/step - loss: 0.2363 - mean_absolute_error: 0.3265 - val_loss: 0.3171 - val_mean_absolute_error: 0.3982
Epoch 54/150
1/1 [==============================] - 0s 254ms/step - loss: 0.2296 - mean_absolute_error: 0.3381 - val_loss: 0.3315 - val_mean_absolute_error: 0.4351
Epoch 55/150
1/1 [==============================] - 0s 246ms/step - loss: 0.2055 - mean_absolute_error: 0.3163 - val_loss: 0.3344 - val_mean_absolute_error: 0.4191
Epoch 56/150
1/1 [==============================] - 0s 244ms/step - loss: 0.2173 - mean_absolute_error: 0.3241 - val_loss: 0.2965 - val_mean_absolute_error: 0.3835
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 283ms/step - loss: 0.2391 - mean_absolute_error: 0.3303 - val_loss: 0.3666 - val_mean_absolute_error: 0.4452
Epoch 102/150
1/1 [==============================] - 0s 263ms/step - loss: 0.2202 - mean_absolute_error: 0.3203 - val_loss: 0.3423 - val_mean_absolute_error: 0.4224
Epoch 103/150
1/1 [==============================] - 0s 252ms/step - loss: 0.1917 - mean_absolute_error: 0.2925 - val_loss: 0.3908 - val_mean_absolute_error: 0.4771
Epoch 104/150
1/1 [==============================] - 0s 226ms/step - loss: 0.2314 - mean_absolute_error: 0.3332 - val_loss: 0.4056 - val_mean_absolute_error: 0.4710
Epoch 105/150
1/1 [==============================] - 0s 200ms/step - loss: 0.2113 - mean_absolute_error: 0.3153 - val_loss: 0.4568 - val_mean_absolute_error: 0.4966
Epoch 106/150
1/1 [==============================] - 0s 203ms/step - loss: 0.1663 - mean_absolute_error: 0.2846 - val_loss: 0.3527 - val_mean_absolute_error: 0.4256
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 85ms/step - loss: 0.3190 - mean_absolute_error: 0.3660
val performance = [0.6936252117156982, 0.584359884262085]
test performance =  [0.31901127099990845, 0.36599746346473694]


In [83]:
plot_history(history)

/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Autoregressive model**

In [80]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

Output shape (batch, time, features):  (128, 120, 1)


In [81]:
it = iter(multi_window.train)
x,y = next(it)
print(x.shape, y.shape)
pred = feedback_model.predict(x)
pred.shape

(128, 168, 18) (128, 120, 1)


(128, 120, 1)

In [82]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test.repeat(-1), steps=100, verbose=1)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

Epoch 1/400
1/1 [==============================] - 8s 8s/step - loss: 0.6060 - mean_absolute_error: 0.5333 - val_loss: 0.2967 - val_mean_absolute_error: 0.4665
Epoch 2/400
1/1 [==============================] - 0s 300ms/step - loss: 0.7439 - mean_absolute_error: 0.6076 - val_loss: 0.3242 - val_mean_absolute_error: 0.4884
Epoch 3/400
1/1 [==============================] - 0s 324ms/step - loss: 0.9033 - mean_absolute_error: 0.6133 - val_loss: 0.3232 - val_mean_absolute_error: 0.4970
Epoch 4/400
1/1 [==============================] - 0s 289ms/step - loss: 0.7884 - mean_absolute_error: 0.5995 - val_loss: 0.3287 - val_mean_absolute_error: 0.5030
Epoch 5/400
1/1 [==============================] - 0s 273ms/step - loss: 0.7690 - mean_absolute_error: 0.6044 - val_loss: 0.3503 - val_mean_absolute_error: 0.5061
Epoch 6/400
1/1 [==============================] - 0s 271ms/step - loss: 0.7607 - mean_absolute_error: 0.5634 - val_loss: 0.3158 - val_mean_absolute_error: 0.4802
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 284ms/step - loss: 0.2414 - mean_absolute_error: 0.3743 - val_loss: 0.2488 - val_mean_absolute_error: 0.3661
Epoch 52/400
1/1 [==============================] - 0s 302ms/step - loss: 0.3477 - mean_absolute_error: 0.4164 - val_loss: 0.2310 - val_mean_absolute_error: 0.3483
Epoch 53/400
1/1 [==============================] - 0s 315ms/step - loss: 0.3371 - mean_absolute_error: 0.3981 - val_loss: 0.2390 - val_mean_absolute_error: 0.3516
Epoch 54/400
1/1 [==============================] - 0s 271ms/step - loss: 0.3251 - mean_absolute_error: 0.4172 - val_loss: 0.2142 - val_mean_absolute_error: 0.3405
Epoch 55/400
1/1 [==============================] - 0s 283ms/step - loss: 0.2403 - mean_absolute_error: 0.3598 - val_loss: 0.2158 - val_mean_absolute_error: 0.3557
Epoch 56/400
1/1 [==============================] - 0s 284ms/step - loss: 0.4424 - mean_absolute_error: 0.4157 - val_loss: 0.2192 - val_mean_absolute_error: 0.3560
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 272ms/step - loss: 0.2517 - mean_absolute_error: 0.3058 - val_loss: 0.1450 - val_mean_absolute_error: 0.2999
Epoch 102/400
1/1 [==============================] - 0s 302ms/step - loss: 0.3049 - mean_absolute_error: 0.3221 - val_loss: 0.1669 - val_mean_absolute_error: 0.2901
Epoch 103/400
1/1 [==============================] - 0s 290ms/step - loss: 0.2585 - mean_absolute_error: 0.3067 - val_loss: 0.1454 - val_mean_absolute_error: 0.2645
Epoch 104/400
1/1 [==============================] - 0s 293ms/step - loss: 0.2531 - mean_absolute_error: 0.3128 - val_loss: 0.1318 - val_mean_absolute_error: 0.2769
Epoch 105/400
1/1 [==============================] - 0s 311ms/step - loss: 0.2827 - mean_absolute_error: 0.3295 - val_loss: 0.1883 - val_mean_absolute_error: 0.3117
Epoch 106/400
1/1 [==============================] - 0s 342ms/step - loss: 0.2021 - mean_absolute_error: 0.2887 - val_loss: 0.1472 - val_mean_absolute_error: 0.2595
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 304ms/step - loss: 0.2265 - mean_absolute_error: 0.3072 - val_loss: 0.1799 - val_mean_absolute_error: 0.3043
Epoch 152/400
1/1 [==============================] - 0s 254ms/step - loss: 0.2768 - mean_absolute_error: 0.3180 - val_loss: 0.1505 - val_mean_absolute_error: 0.2798
Epoch 153/400
1/1 [==============================] - 0s 279ms/step - loss: 0.2447 - mean_absolute_error: 0.2958 - val_loss: 0.1508 - val_mean_absolute_error: 0.2872
Epoch 154/400
1/1 [==============================] - 0s 253ms/step - loss: 0.2628 - mean_absolute_error: 0.2938 - val_loss: 0.1389 - val_mean_absolute_error: 0.2941
Epoch 155/400
1/1 [==============================] - 0s 313ms/step - loss: 0.2543 - mean_absolute_error: 0.3027 - val_loss: 0.1212 - val_mean_absolute_error: 0.2748
Epoch 156/400
1/1 [==============================] - 0s 332ms/step - loss: 0.2099 - mean_absolute_error: 0.2998 - val_loss: 0.1651 - val_mean_absolute_error: 0.3126
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 279ms/step - loss: 0.1611 - mean_absolute_error: 0.2747 - val_loss: 0.1344 - val_mean_absolute_error: 0.2803
Epoch 202/400
1/1 [==============================] - 0s 284ms/step - loss: 0.1203 - mean_absolute_error: 0.2392 - val_loss: 0.1309 - val_mean_absolute_error: 0.2691
Epoch 203/400
1/1 [==============================] - 0s 256ms/step - loss: 0.2032 - mean_absolute_error: 0.3000 - val_loss: 0.1867 - val_mean_absolute_error: 0.2802
Epoch 204/400
1/1 [==============================] - 0s 272ms/step - loss: 0.2181 - mean_absolute_error: 0.2887 - val_loss: 0.1706 - val_mean_absolute_error: 0.2638
Epoch 205/400
1/1 [==============================] - 0s 291ms/step - loss: 0.2673 - mean_absolute_error: 0.3055 - val_loss: 0.1367 - val_mean_absolute_error: 0.2765
Epoch 206/400
1/1 [==============================] - 0s 280ms/step - loss: 0.2296 - mean_absolute_error: 0.2837 - val_loss: 0.1283 - val_mean_absolute_error: 0.2691
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 275ms/step - loss: 0.1949 - mean_absolute_error: 0.2839 - val_loss: 0.1358 - val_mean_absolute_error: 0.2821
Epoch 252/400
1/1 [==============================] - 0s 302ms/step - loss: 0.2243 - mean_absolute_error: 0.2991 - val_loss: 0.1249 - val_mean_absolute_error: 0.2602
Epoch 253/400
1/1 [==============================] - 0s 300ms/step - loss: 0.1800 - mean_absolute_error: 0.2633 - val_loss: 0.1138 - val_mean_absolute_error: 0.2517
Epoch 254/400
1/1 [==============================] - 0s 275ms/step - loss: 0.1664 - mean_absolute_error: 0.2674 - val_loss: 0.1570 - val_mean_absolute_error: 0.2564
Epoch 255/400
1/1 [==============================] - 0s 303ms/step - loss: 0.1790 - mean_absolute_error: 0.2734 - val_loss: 0.1335 - val_mean_absolute_error: 0.2661
Epoch 256/400
1/1 [==============================] - 0s 284ms/step - loss: 0.1948 - mean_absolute_error: 0.2898 - val_loss: 0.1656 - val_mean_absolute_error: 0.2617
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 324ms/step - loss: 0.1967 - mean_absolute_error: 0.2591 - val_loss: 0.1550 - val_mean_absolute_error: 0.2819
Epoch 302/400
1/1 [==============================] - 0s 312ms/step - loss: 0.2262 - mean_absolute_error: 0.2799 - val_loss: 0.1378 - val_mean_absolute_error: 0.2619
Epoch 303/400
1/1 [==============================] - 0s 268ms/step - loss: 0.1592 - mean_absolute_error: 0.2785 - val_loss: 0.1277 - val_mean_absolute_error: 0.2602
Epoch 304/400
1/1 [==============================] - 0s 313ms/step - loss: 0.1957 - mean_absolute_error: 0.2838 - val_loss: 0.1094 - val_mean_absolute_error: 0.2423
Epoch 305/400
1/1 [==============================] - 0s 284ms/step - loss: 0.2626 - mean_absolute_error: 0.3006 - val_loss: 0.2344 - val_mean_absolute_error: 0.2968
Epoch 306/400
1/1 [==============================] - 0s 312ms/step - loss: 0.2530 - mean_absolute_error: 0.3080 - val_loss: 0.1155 - val_mean_absolute_error: 0.2555
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 283ms/step - loss: 0.1115 - mean_absolute_error: 0.2239 - val_loss: 0.1440 - val_mean_absolute_error: 0.2650
Epoch 352/400
1/1 [==============================] - 0s 266ms/step - loss: 0.1509 - mean_absolute_error: 0.2292 - val_loss: 0.0967 - val_mean_absolute_error: 0.2310
Epoch 353/400
1/1 [==============================] - 0s 303ms/step - loss: 0.2421 - mean_absolute_error: 0.2934 - val_loss: 0.1587 - val_mean_absolute_error: 0.2793
Epoch 354/400
1/1 [==============================] - 0s 291ms/step - loss: 0.2195 - mean_absolute_error: 0.2908 - val_loss: 0.1113 - val_mean_absolute_error: 0.2552
Epoch 355/400
1/1 [==============================] - 0s 308ms/step - loss: 0.1257 - mean_absolute_error: 0.2379 - val_loss: 0.1363 - val_mean_absolute_error: 0.2789
Epoch 356/400
1/1 [==============================] - 0s 318ms/step - loss: 0.1951 - mean_absolute_error: 0.2708 - val_loss: 0.1146 - val_mean_absolute_error: 0.2622
Epoch 357/

100/100 [==============================] - 9s 91ms/step - loss: 0.1572 - mean_absolute_error: 0.2543
val performance = [0.14866141974925995, 0.268552303314209]
test performance =  [0.1571728140115738, 0.2542666494846344]


In [83]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
multi_window.plot2(feedback_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**performance**

In [325]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [326]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.296343,-0.974057,2.019218,0.607528,-0.242254,-1.148407,-0.529166,-1.999581e-12,1.414204,-0.003771,1.414155,-1.274197,-0.205486,1.969427,0.334400,1.266422,-1.613266,-0.773232
1,-1.296343,-0.974057,1.994322,0.607528,-0.250365,-1.032192,-0.521539,3.660228e-01,1.366016,-0.002757,1.414157,-1.274197,-0.205486,1.969427,0.334400,1.266422,-1.729481,-0.765605
2,-1.296343,-0.974057,1.994322,0.568510,-0.250365,-1.032192,-0.529166,7.071018e-01,1.224736,-0.001743,1.414159,-1.274197,-0.205486,1.969427,0.295382,1.274533,-1.729481,-0.750351
3,-1.296343,-0.974057,1.994322,0.568510,-0.250365,-1.148407,-0.506285,9.999928e-01,0.999993,-0.000730,1.414160,-1.274197,-0.205486,1.969427,0.295382,1.266422,-1.729481,-0.750351
4,-1.296343,-0.974057,2.044113,0.568510,-0.250365,-1.032192,-0.521539,1.224736e+00,0.707102,0.000284,1.414160,-1.274197,-0.205486,1.969427,0.295382,1.266422,-1.613266,-0.742724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,-1.218831,0.178799,0.052464,0.997712,-0.436922,-0.218688,0.111505,-1.366016e+00,0.366023,-0.032435,1.413788,-1.307416,0.563084,0.525480,-0.016765,-0.331476,-0.683548,3.528421
35036,-1.218831,0.178799,0.052464,0.997712,-0.436922,-0.218688,0.111505,-1.224736e+00,0.707102,-0.031422,1.413811,-1.318489,0.563084,0.450794,-0.016765,-0.315254,-0.683548,3.421643
35037,-1.229905,0.178799,0.077359,0.997712,-0.436922,-0.334903,0.088624,-9.999928e-01,0.999993,-0.030408,1.413833,-1.318489,0.563084,0.326315,-0.016765,-0.307143,-0.683548,3.421643
35038,-1.229905,0.178799,0.077359,0.997712,-0.445033,-0.218688,0.080997,-7.071018e-01,1.224736,-0.029395,1.413854,-1.318489,0.563084,0.326315,-0.055783,-0.323365,-0.596513,3.444524


In [327]:
test_df.shape

(35040, 18)

In [328]:
from tqdm import tqdm

class DaysPrediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=24*5,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        len = df.shape[0]//in_steps*in_steps
        self.n_days = (len - in_steps)//24
        self.pred_days = out_steps//24
        self.in_days = in_steps//24
        self.gt = np.full((self.n_days), np.nan)
        for i in range(self.n_days):
            x = test_df[i*24:(i*24+24)].to_numpy()
            self.gt[i] = np.average(x[0:24, out_features[0]])
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days), np.nan)
        for i in tqdm(range(self.n_days-self.in_days)):
            x = test_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][i+self.in_days+j] = np.average(y[:, j*24:j*24+24, :])
        return preds

In [329]:
preds_dict = {}
days_prediction = DaysPrediction()
preds_dict['GRU'] = days_prediction.predict(multi_gru_model)

100%|██████████| 1450/1450 [01:19<00:00, 18.25it/s]


In [330]:
def denormalize(data):
    return data*train_std[out_features[0]]+train_mean[out_features[0]]

In [331]:
gt = denormalize(days_prediction.gt)
denorm_preds = {}
denorm_preds['GRU'] = denormalize(preds_dict['GRU'])

In [332]:
plt.figure(figsize=(8, 3))
plt.plot(gt)
plt.plot(denorm_preds['GRU'][4])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [343]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae = np.mean(np.absolute(gt[pred_slice] - (denorm_preds['GRU'][pred_day])[pred_slice]))
print(mae)
mape = np.mean(np.absolute((gt[pred_slice] - denorm_preds['GRU'][pred_day][pred_slice])/gt[pred_slice]))
print(mape)

2.5811294367907194
0.2368160687057402


0

In [334]:
quality_standard = [
    [], #0
    [],
    [],
    [],
    [],
    {'dir':'lte', 'level': [2,3,4,5,6,8]}, #Cl
    {'dir':'lte', 'level': [5,9,14,20,35,70]}, #Cl
]



In [335]:
def calc_level(feature_id, data):
    standard = quality_standard[feature_id]
    levels = np.empty((0,))
    n_level = len(standard['level'])
    if (standard['dir']=='lte'):
        for i in range(len(data)):
            level = n_level
            for j in range(n_level):
                if data[i] <= standard['level'][j]:
                    level = j
                    break
            levels = np.append(levels, level)
    return levels

In [336]:
gt_level = calc_level(out_features[0], gt[pred_slice])
pred_level = calc_level(out_features[0], denorm_preds['GRU'][4][pred_slice])
print('precision =', np.mean((gt_level == pred_level).astype(float))*100)

precision = 44.12171507607192


In [236]:
gt_level[0:100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [237]:
test_df.shape

INPUT_STEPS = 24*5
len = test_df.shape[0]//INPUT_STEPS*INPUT_STEPS
n_days = (len - INPUT_STEPS)//24
pred_days = OUT_STEPS//24
input_days = INPUT_STEPS//24
print('n_days=', n_days, 'pred_days = ', pred_days)

preds = np.zeros((pred_days, n_days))
gt = np.zeros((n_days))

for i in range(n_days-input_days-pred_days):#for i in range(100):
    x = test_df[i*24:(i*24+INPUT_STEPS)].to_numpy()
    gt[i] = np.average(x[0:24, out_features[0]])
    x = x.reshape((-1, INPUT_STEPS, test_df.shape[1]))
    y = multi_lstm_model(x)
    for j in range(pred_days):
        preds[j][i+input_days+j] = np.average(y[:, j*24:j*24+24, :])

n_days= 1455 pred_days =  5


In [344]:

plt.figure()
plt.plot(gt)
plt.plot(preds[4])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
OUT_STEPS = 24*5
seq_window = WindowGenerator(input_width=24*5,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

seq_window.plot2(plot_col=out_features[0])
seq_window

## 연습 섹션

In [ ]:
it = iter(wide_window.val)
x,y = next(it)

In [ ]:
x.shape, y.shape

In [ ]:
history = gain.fit(wide_window.train, epochs=20,
                      validation_data=wide_window.val,
                      callbacks=[])

In [ ]:
it = iter(wide_window.val)
x,y = next(it)
x.shape, y.shape

In [ ]:
gain.predict(x)

In [ ]:
df[0].isna().astype(int).sum()

In [ ]:
date_time1 = pd.to_datetime(df_full[0].iloc[:, 0], format='%Y.%m.%d %H:%M')
date_time2 = pd.to_datetime(df_full[0].iloc[:, 0], format='%Y.%m.%d %H:%M')

In [ ]:
timestamp_s1 = date_time1.map(datetime.datetime.timestamp)
timestamp_s2 = date_time2.map(datetime.datetime.timestamp)

In [ ]:
day = 24*60*60
year = (365.2425)*day

df[0]['Day sin'] = np.sin(timestamp_s1 * (2 * np.pi / day))
df[0]['Day cos'] = np.cos(timestamp_s1 * (2 * np.pi / day))
df[0]['Year sin'] = np.sin(timestamp_s1 * (2 * np.pi / year))
df[0]['Year cos'] = np.cos(timestamp_s1 * (2 * np.pi / year))

df[1]['Day sin'] = np.sin(timestamp_s2 * (2 * np.pi / day))
df[1]['Day cos'] = np.cos(timestamp_s2 * (2 * np.pi / day))
df[1]['Year sin'] = np.sin(timestamp_s2 * (2 * np.pi / year))
df[1]['Year cos'] = np.cos(timestamp_s2 * (2 * np.pi / year))

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        print(data[0].shape)
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((x,y))

In [ ]:
ds.element_spec


In [ ]:
ds = ds.batch(5)
ds.element_spec

In [ ]:
df2

In [ ]:
model.fit(ds)

# MNIST with data generator

https://towardsdatascience.com/keras-custom-data-generators-example-with-mnist-dataset-2a7a2d2b0360


In [ ]:
import tensorflow as tf
import os
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
import math

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
 
    def __init__(self, X_data , y_data, batch_size, dim, n_classes,
                 to_fit, shuffle = True):
        self.batch_size = batch_size
        self.X_data = X_data
        self.labels = y_data
        self.y_data = y_data
        self.to_fit = to_fit
        self.n_classes = n_classes
        self.dim = dim
        self.shuffle = shuffle
        self.n = 0
        self.list_IDs = np.arange(len(self.X_data))
        self.on_epoch_end()
    def __next__(self):
        # Get one batch of data
        data = self.__getitem__(self.n)
        # Batch index
        self.n += 1
        
        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end
            self.n = 0
        
        return data
    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:
            (index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(list_IDs_temp)
        
        if self.to_fit:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.X_data))
        
        if self.shuffle: 
            np.random.shuffle(self.indexes)
    def _generate_x(self, list_IDs_temp):
               
        X = np.empty((self.batch_size, *self.dim))
        
        for i, ID in enumerate(list_IDs_temp):
            
            X[i,] = self.X_data[ID]
            
            # Normalize data
            X = (X/255).astype('float32')
            
        return X[:,:,:, np.newaxis]
    def _generate_y(self, list_IDs_temp):
        
        y = np.empty(self.batch_size)
        
        for i, ID in enumerate(list_IDs_temp):
            
            y[i] = self.y_data[ID]
            
        return keras.utils.to_categorical(
                y,num_classes=self.n_classes)

In [ ]:
n_classes = 10
input_shape = (28, 28)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28 , 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
train_generator = DataGenerator(x_train, y_train, batch_size = 64,
                                dim = input_shape,
                                n_classes=10, 
                                to_fit=True, shuffle=True)
val_generator =  DataGenerator(x_test, y_test, batch_size=64, 
                               dim = input_shape, 
                               n_classes= n_classes, 
                               to_fit=True, shuffle=True)

In [ ]:
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

In [ ]:
model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=1,
        validation_data=val_generator,
        validation_steps=validation_steps)

In [ ]:
it = iter(train_generator)

In [ ]:
x,y = next(it)

In [ ]:
x.shape

In [ ]:
y.shape

## MNIST with custom model

In [ ]:
n_classes = 10
input_shape = (28, 28, 1)
input_data = keras.layers.Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3),
                 activation='relu')(input_data)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_data = Dense(n_classes, activation='softmax')(x)
model = CustomModel(input_data, output_data)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=1,
        validation_data=val_generator,
        validation_steps=validation_steps)

결론: DataGenerator 만으로는 train_step에 input data의 shape에 None으로 들어간다.

```py
        X = keras.layers.Reshape((tf.reduce_sum(x.shape[1:]),))(x)
        Y = keras.layers.Reshape((tf.reduce_sum(x.shape[1:]),))(y)
        X = tf.reshape(x, shape=(x.shape[0], -1))
        Y = tf.reshape(y, shape=(x.shape[0], -1)
```

이런 함수들을 train_step 내에 사용할 수 없다

# 한글 폰트

In [ ]:


import matplotlib
import matplotlib.font_manager

[f.fname for f in matplotlib.font_manager.fontManager.ttflist]



In [ ]:
! fc-list :lang=ko

In [ ]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [ ]:
fig = plt.figure()  
plt.plot((1,1), label='가-가가')  
plt.title('가가가',fontproperties=fprop)  
plt.legend(prop=fprop)  
plt.show()